In [ ]:
# raw_ciha 
# Este notebook foi utilizado para transformar a base do CIHA de csv para PARQUET, criando a camada raw

In [1]:
%run ../spark-default.py

In [2]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [4]:
# Diretório no S3
s3_path = "s3a://datalake/source/ciha_csv/"

# Faz a listagem dos arquivos no diretório
c = spark.read.csv(s3_path).withColumn("input_file_name", input_file_name()) 
print(c.select("input_file_name").distinct().count())

4411


In [3]:
ciha_raw = spark.read.option("header", True).csv("s3a://datalake/source/ciha_csv/")
    
ciha = ciha_raw.select(
    col("ANO_CMPT").alias("ano_cmpt"),
    col("MES_CMPT").alias("mes_cmpt"),
    col("ESPEC").alias("espec"),
    col("CGC_HOSP").alias("cgc_hosp"),
    col("MUNIC_RES").alias("munic_res"),
    to_date("NASC", "yyyyMMdd").alias("nasc"),
    col("SEXO").alias("sexo"),
    col("UTI_MES_TO").alias("uti_mes_to"),
    col("UTI_INT_TO").alias("uti_int_to"),
    col("PROC_REA").alias("proc_rea"),
    col("QT_PROC").alias("qt_proc"),
    to_date("DT_ATEND", "yyyyMMdd").alias("dt_atend"),
    to_date("DT_SAIDA", "yyyyMMdd").alias("dt_saida"),
    col("DIAG_PRINC").alias("diag_princ"),
    col("DIAG_SECUN").alias("diag_secun"),
    col("COBRANCA").alias("cobranca"),
    col("NATUREZA").alias("natureza"),
    col("GESTAO").alias("gestao"),
    col("MUNIC_MOV").alias("munic_mov"),
    col("COD_IDADE").alias("cod_idade"),
    col("IDADE").alias("idade"),
    col("DIAS_PERM").alias("dias_perm"),
    col("MORTE").alias("morte"),
    col("NACIONAL").alias("nacional"),
    col("CAR_INT").alias("car_int"),
    col("HOMONIMO").alias("homonimo"),
    col("CNES").alias("cnes"),
    col("FONTE").alias("fonte"),
    col("CGC_CONSOR").alias("cgc_consor"),
    col("MODALIDADE").alias("modalidade"),
    input_file_name().alias("input_file_name")
).withColumn("estado_uf", substring( split(col("input_file_name"), "/")[5], 5, 2))

ciha.printSchema()

# print(ciha.count())

# ciha.show()

root
 |-- ano_cmpt: string (nullable = true)
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- homon

In [4]:
ciha.groupBy("ano_cmpt").agg(count("*")).orderBy("ano_cmpt").show(truncate=False)

+--------+--------+
|ano_cmpt|count(1)|
+--------+--------+
|2011    |10642684|
|2012    |12857677|
|2013    |14972511|
|2014    |18552160|
|2015    |16636260|
|2016    |18869126|
|2017    |20354660|
|2018    |18750287|
|2019    |19629407|
|2020    |15326424|
|2021    |18579192|
|2022    |20090317|
|2023    |20297787|
|2024    |17560671|
|2025    |4641932 |
+--------+--------+



In [5]:
ciha.groupBy("estado_uf").agg(count("*")).orderBy("estado_uf").show(truncate=False)

+---------+--------+
|estado_uf|count(1)|
+---------+--------+
|AC       |50888   |
|AL       |1541638 |
|AM       |1352789 |
|AP       |843993  |
|BA       |10254201|
|CE       |2803868 |
|DF       |144919  |
|ES       |2637869 |
|GO       |1241902 |
|MA       |430759  |
|MG       |30222301|
|MS       |1634878 |
|MT       |798926  |
|PA       |4353018 |
|PB       |534198  |
|PE       |5495413 |
|PI       |525618  |
|PR       |16419389|
|RJ       |9273994 |
|RN       |993615  |
+---------+--------+
only showing top 20 rows



In [ ]:
# NASC DT_ATEND DT_SAIDA

ciha.filter("ano_cmpt = '2023'").select(
    min("NASC").alias("min_NASC"),
    min("DT_ATEND").alias("min_DT_ATEND"),
    min("DT_SAIDA").alias("min_DT_SAIDA")
)

In [8]:
ciha.filter("ano_cmpt = '2023' and nasc < '1500-01-01'")

ano_cmpt,mes_cmpt,espec,cgc_hosp,munic_res,nasc,sexo,uti_mes_to,uti_int_to,proc_rea,qt_proc,dt_atend,dt_saida,diag_princ,diag_secun,cobranca,natureza,gestao,munic_mov,cod_idade,idade,dias_perm,morte,nacional,car_int,homonimo,cnes,fonte,cgc_consor,modalidade,input_file_name,estado_uf
2023,01,null,92815000000834,043090,0196-06-01,0,0,000,0209020016,1,2023-01-04,2023-01-04,C670,null,18,null,M,430920,0,99,0,0,null,null,null,2232049,06,00000000000000,01,s3a://datalake/so...,RS
2023,08,null,76591049000128,042054,0195-10-05,0,0,000,0301010072,1,2023-08-14,2023-08-14,null,null,18,null,D,410690,0,99,0,0,null,null,null,0015644,01,00000000000000,01,s3a://datalake/so...,PR
2023,08,null,76591049000128,042054,0195-10-05,0,0,000,0301010072,2,2023-08-09,2023-08-09,null,null,18,null,D,410690,0,99,0,0,null,null,null,0015644,01,00000000000000,01,s3a://datalake/so...,PR
2023,08,null,76591049000128,042054,0195-10-05,0,0,000,0301010072,1,2023-08-11,2023-08-11,null,null,18,null,D,410690,0,99,0,0,null,null,null,0015644,01,00000000000000,01,s3a://datalake/so...,PR
2023,08,null,76591049000128,042054,0195-10-05,0,0,000,0301010072,1,2023-08-04,2023-08-04,null,null,18,null,D,410690,0,99,0,0,null,null,null,0015644,01,00000000000000,01,s3a://datalake/so...,PR
2023,08,null,76591049000128,042054,0195-10-05,0,0,000,0211020036,1,2023-08-02,2023-08-02,null,null,18,null,D,410690,0,99,0,0,null,null,null,0015644,01,00000000000000,01,s3a://datalake/so...,PR
2023,08,null,76591049000128,042054,0195-10-05,0,0,000,0301010072,1,2023-08-02,2023-08-02,null,null,18,null,D,410690,0,99,0,0,null,null,null,0015644,01,00000000000000,01,s3a://datalake/so...,PR


In [6]:
for ano in [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024, 2025]:
    print(ano)
    ciha.filter(f"ano_cmpt = '{ano}'").filter("nasc >= '1500-01-01' ").write.mode("overwrite").partitionBy("ano_cmpt").parquet("s3a://datalake/raw/ciha/")

2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025


In [7]:
spark.stop()